In [38]:
#Import the neccessary libraries and functions 
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import os
import glob
from matplotlib import rcParams
from pandas import Series, DataFrame
import seaborn as sb
import csv
import datetime as datetime
from datetime import datetime
import warnings
from tkinter import filedialog
from tkinter import *
warnings.filterwarnings('ignore')

from degree_days import bug_degree_day, plant_degree_day
from time_of_day import hot_cold_time
from data_readin import data_read_in
from qa_qc_tests import persist_test, range_test, step_test

#To change the figure size you can use below parameters.
rcParams['figure.figsize'] = 10, 8 
sb.set_style('whitegrid')


In [33]:
#Read in the directory using an interactive interface
#root = Tk()
#root.withdraw()
#folder_selected = filedialog.askdirectory()
#path = folder_selected
path = '/Users/mgrover1/USGS_test/'
extension = 'csv'
os.chdir(path)
result = [i for i in glob.glob('*.{}'.format(extension))]
print(result)


['yearly_avgtemp_report.csv', 'T197198_9871281pole.csv', 'Large_Differences_Between_Readings.csv', 'full_dataframe.csv', 'T197198_9926710pole.csv', 'T6667_9926779tube.csv', 'TI23_9926761pole.csv', 'T6667_9926756ground.csv', 'TI23_9926732tube.csv', 'weekly_avgtemp_report.csv', 'monthly_count_report.csv', 'yearly_count_report.csv', 'monthly_avgtemp_report.csv', 'TI21_9926653tube.csv', 'TI23_10140293ground.csv', 'TI30_9926658tube.csv', 'output_report.csv', 'TI30_10140219pole.csv', 'weekly_count_report.csv']


In [39]:
#Complete the data read in process by pulling the data into a single dataframe
#Exports the dataframe to its own file 
x = data_read_in(result)


NameError: name 'result' is not defined

In [20]:
#QA/QC Analysis 

#PERSISTENCE TEST
#Checks to see if there are too many values in a row
persist_df_result = persist_test(temps, 10, 2)
x['persist_test'] = persist_df_result['result']

#RANGE TEST
#Set the maximum and minimum temperature values you would expect from the sensor 
range_test_df = range_test(temps,90,5)
x['range_test']= range_test_df['result']

#STEP TEST
#Run the step test, with the second value being the maximum difference you would expect between time steps
step_test_df = step_test(temps,5)
x['step_test'] = step_test_df['result']

#Tests to see if each timestep satisfies all the QA/QC components 
all_tests = []
for i in range(len(x)):
    if x['step_test'][i] & x['range_test'][i] & x['persist_test'][i] == True:
        all_tests.append('Pass')
    else:
        all_tests.append("Fail")
x['all_tests'] = all_tests



In [ ]:
#Calculate different degree days 
x['Plant_Degree_Day'] = plant_degree_day(x)
x['Bug_Degree_Day'] = bug_degree_day(x)

In [ ]:
#Create reports detailing the number of temperature observations for each given sensor 
#Yearly Report 

#Create the yearly report for both count and average temperature 
yearly_report_count = x.groupby(['site','type','Year'])['temp'].count()
yearly_report_mean = x.groupby(['site','type','Year'])['temp'].mean()

#Exports the yearly reports
yearly_report_count.to_csv('yearly_count_report.csv',header=True)
yearly_report_mean.to_csv('yearly_avgtemp_report.csv',header=True)


#Create the monthly reports for both count and average temperature 
monthly_report_count = x.groupby(['site','type','Year','Month'])['temp'].count()
monthly_report_mean = x.groupby(['site','type','Year','Month'])['temp'].mean()

#Exports the monthly reports 
monthly_report_count.to_csv('monthly_count_report.csv',header=True)
monthly_report_mean.to_csv('monthly_avgtemp_report.csv',header=True)


#Create the weekly report for both count and average temperature 
weekly_report_count = x.groupby(['site','type','Year','Month','Week'])['temp'].count()
weekly_report_mean = x.groupby(['site','type','Year','Month','Week'])['temp'].mean()

#Creates the reports 
weekly_report_count.to_csv('weekly_count_report.csv',header=True)
weekly_report_mean.to_csv('weekly_avgtemp_report.csv',header=True)

#Create the weekly report for both count and sum of plant/bug degree days

weekly_report_sum_plant = x.groupby(['site','type','Year','Month','Week'])['Plant_Degree_Day'].sum()
weekly_report_sum_bug = x.groupby(['site','type','Year','Month','Week'])['Bug_Degree_Day'].sum()

#Creates the reports 
weekly_report_sum_plant.to_csv('weekly_sum_plant_report.csv',header=True)
weekly_report_sum_bug.to_csv('weekly_sum_bug_report.csv',header=True)
